# Tobig's 14기 2주차 Optimization 과제_ 14기 이혜린
### Made by 이지용

# Gradient Descent 구현하기

### 1) "..." 표시되어 있는 빈 칸을 채워주세요  
### 2) 강의내용과 코드에 대해 공부한 내용을 적어서 과제를 채워주세요

In [49]:
import pandas as pd
import numpy as np
import random

In [50]:
data = pd.read_csv('assignment_2.csv')
data.head()

,Label,bias,experience,salary
0,1,1,0.7,48000
1,0,1,1.9,48000
2,1,1,2.5,60000
3,0,1,4.2,63000
4,0,1,6.0,76000


In [51]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Label       200 non-null    int64  
 1   bias        200 non-null    int64  
 2   experience  200 non-null    float64
 3   salary      200 non-null    int64  
dtypes: float64(1), int64(3)
memory usage: 6.4 KB


## Train Test 데이터 나누기
### 데이터셋을 train/test로 나눠주는 메소드  
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [52]:
from sklearn.model_selection import train_test_split

In [53]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0], test_size=0.25, random_state = 0)

In [54]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((150, 3), (50, 3), (150,), (50,))

## Scaling  

experience와 salary의 단위, 평균, 분산이 크게 차이나므로 scaler를 사용해 단위를 맞춰줍니다. 

In [55]:
#경사하강법을 사용할 때는 반드시 모든 특성이 같은 스케일을 갖도록 만들어야 한다. 그래야 수렴하는 데 짧은 시간이 걸린다.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
bias_train = X_train["bias"]
bias_train = bias_train.reset_index()["bias"]
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_train["bias"] = bias_train
X_train.head()

,bias,experience,salary
0,1,0.187893,-1.143335
1,1,1.185555,0.043974
2,1,-0.310938,-0.351795
3,1,-1.629277,-1.341220
4,1,-1.308600,0.043974


이때 scaler는 X_train에 fit 해주시고, fit한 scaler를 X_test에 적용시켜줍니다.  
똑같이 X_test에다 fit하면 안돼요!

In [56]:
bias_test = X_test["bias"]
bias_test = bias_test.reset_index()["bias"]
X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
X_test["bias"] = bias_test
X_test.head()

,bias,experience,salary
0,1,-1.344231,-0.615642
1,1,0.508570,0.307821
2,1,-0.310938,0.571667
3,1,1.363709,1.956862
4,1,-0.987923,-0.747565


In [57]:
# parameter 개수
N = len(X_train.loc[0])

In [58]:
# 초기 parameter들을 임의로 설정해줍니다.
parameters = np.array([random.random() for i in range(N)])
parameters

array([0.0648676 , 0.10798919, 0.58104953])

### * LaTeX   

Jupyter Notebook은 LaTeX 문법으로 수식 입력을 지원하고 있습니다.  
http://triki.net/apps/3466  
https://jjycjnmath.tistory.com/117

## Logistic Function

## $p = {1 \over {1 + e^{-(\beta_{0} + \beta_{1}x)}}}$

In [59]:
def logistic(X, parameters):
    z = 0
    for i in range(len(parameters)) :
        z += X[i] * parameters[i]
    p = 1 / (1 + np.exp(-z))
    
    return p

In [60]:
logistic(X_train.iloc[1], parameters)

0.5543953797819545

## Object Function

Object Function : 목적함수는 Gradient Descent를 통해 최적화 하고자 하는 함수입니다.  
로지스틱 회귀의 목적함수를 작성해주세요
## $l(p) = -{1 \over N} \sum_{i=1}^{n}{[y_{i}log(p_{i}) + (1-y_{i})log(1-p_{i})]}$  
  
+ 전체 데이터를 사용하는 batch gradient descent의 경우 N으로 나누어 주는 것이 맞다.

In [61]:
def cross_entropy_i(X, y, parameters) :
    p = logistic(X, parameters)                            # 위에서 작성한 함수를 활용하세요
    loss = y * np.log(p) + (1-y) * np.log(1-p)
    return -loss

In [62]:
def cross_entropy(X_set, y_set, parameters) :
    final_loss = 0  
    for i in range(X_set.shape[0]):
        X = X_set.iloc[i, :]
        y = y_set.iloc[i] 
        final_loss += cross_entropy_i(X, y, parameters)
    return final_loss / X_set.shape[0]

In [63]:
cross_entropy(X_test, y_test, parameters)

0.875155326297531

## Gradient of Cross Entropy

## ${\partial\over{\partial \theta_j}}l(p)= -{1 \over N}\sum_{}^{}{(y_{i}-p_{i})x_{ij}}$  
  
+ 전체 데이터를 사용하는 batch gradient descent의 경우 N으로 나누어 주는 것이 맞다.

In [64]:
# cross_entropy를 theta_j에 대해 미분한 값을 구하는 함수
def get_gradient_ij_cross_entropy(X, y, parameters, j):
    p = logistic(X, parameters)
    gradient = (y-p) * X[j]
    return -gradient

In [65]:
get_gradient_ij_cross_entropy(X_train.iloc[0, :], y_train.iloc[0], parameters, 1)

-0.12041648503583766

## Batch Gradient Descent  

Batch Gradient Descent : 학습 한번에 전체 데이터에 대해서 기울기(=Gradient)를 구한다.

In [66]:
def get_gradients_bgd(X_train, y_train, parameters) :
    gradients = [0 for i in range(len(parameters))]
    
    for i in range(X_train.shape[0]):
        X = X_train.iloc[i, :]
        y = y_train.iloc[i]
        for j in range(len(parameters)):
            gradients[j] += get_gradient_ij_cross_entropy(X, y, parameters,j)
            
    return gradients

In [67]:
gradients_bgd = get_gradients_bgd(X_train, y_train, parameters)
gradients_bgd

[35.01134615236157, 0.5334126389870246, 36.69361185525045]

## Stochastic Gradient Descent  

Stochastic Gradient Descent : 학습 한번에 임의의 데이터 하나에 대해서만 기울기(=Gradient)를 구한다.

In [68]:
def get_gradients_sgd(X_train, y, parameters) :
    gradients = [0 for i in range(len(parameters))]
    r = int(random.random()*X_train.shape[0])
    X = X_train.iloc[r, :]
    y = y_train.iloc[r]
        
    for j in range(len(parameters)):
        gradients[j] = get_gradient_ij_cross_entropy(X, y, parameters,j)
        
    return gradients

In [69]:
gradients_sgd = get_gradients_sgd(X_train, y_train, parameters)
gradients_sgd

[0.3849014493366882, -0.6271112052490097, -0.2369614592228488]

## Update Parameters  

In [70]:
def update_parameters(parameters, gradients, learning_rate) :
    for i in range(len(parameters)) :
        gradients[i] *= learning_rate
    parameters -= gradients
    return parameters

In [71]:
update_parameters(parameters, gradients_bgd, 0.01)

array([-0.28524586,  0.10265506,  0.21411341])

## Gradient Descent  

위에서 작성한 함수들을 조합해서 Gradient Descent를 진행하는 함수를 완성해주세요

learning_rate = 학습 시 스텝의 크기. 학습률이 너무 크면 학습시간이 적게 걸리지만 global minimum에서 멀어질 수 있고, 학습률이 너무 작으면 학습시간이 많이 걸리고 local minimum으로 갈 확률이 커진다.  
max_iter = 최대 반복 횟수  
tolerance = 허용오차. 허용오차보다 작아지면 (loss가 수렴하면) 거의 최솟값에 도달한 것이므로 알고리즘을 중지한다.

In [72]:
def gradient_descent(X_train, y_train, learning_rate=0.01, max_iter=100000, tolerance=0.0001, optimizer="bgd") :
    count = 1
    point = 100 if optimizer == "bgd" else 10000
    N = len(X_train.iloc[0])
    parameters = np.array([random.random() for i in range(N)])
    gradients = [0 for i in range(N)]
    loss = 0
    
    while count < max_iter :
        
        if optimizer == "bgd" :
            gradients = get_gradients_bgd(X_train, y_train, parameters)
        elif optimizer == "sgd" :
            gradients = get_gradients_sgd(X_train, y_train, parameters)
            # loss, 중단 확인
        if count%point == 0 :
            new_loss = cross_entropy(X_train, y_train, parameters)
            print(count, "loss: ",new_loss, "params: ", parameters, "gradients: ", gradients)
            
            #중단 조건
            if abs(new_loss-loss) < tolerance * len(y_train):
                break
            loss = new_loss
                
            
                
        parameters = update_parameters(parameters, gradients, learning_rate)
        count += 1
    return parameters

In [73]:
new_param_bgd = gradient_descent(X_train, y_train)
new_param_bgd

100 loss:  0.3025314684038491 params:  [-1.62459391  3.47671109 -3.30286204] gradients:  [0.27612190098484074, -0.9085572940474076, 0.8492234715644229]
200 loss:  0.29866751650927764 params:  [-1.78215355  3.99169626 -3.78166211] gradients:  [0.0831724535161957, -0.2702685072489314, 0.24994563830009592]


array([-1.78215355,  3.99169626, -3.78166211])

## Hyper Parameter Tuning

Hyper Parameter들을 매번 다르게 해서 학습을 진행해 보세요. 다른 점들을 발견할 수 있습니다.

In [74]:
new_param_sgd = gradient_descent(X_train, y_train, learning_rate=0.01, max_iter=100000, tolerance=0.0001, optimizer="sgd")
new_param_sgd

10000 loss:  0.30893490316226996 params:  [-1.4559263   3.07553961 -2.97752014] gradients:  [-0.20414755049894406, -0.2347542521799269, -0.0493749872309641]
20000 loss:  0.3002360819268297 params:  [-1.77593624  3.72314268 -3.58982074] gradients:  [-0.7826748332506974, -0.28649577363378254, -0.18929720098674427]


array([-1.77593624,  3.72314268, -3.58982074])

Batch Gradient Descent와 Stochastic Gradient Descent의 결과(추정된 모수)가 같다.

In [75]:
new_param_bgd1 = gradient_descent(X_train, y_train, tolerance = 0.01) #tolerance를 높임
new_param_bgd1

100 loss:  0.3024072546157307 params:  [-1.62779052  3.48722906 -3.31269299] gradients:  [0.271668272847243, -0.8936139460304258, 0.8350683186577292]


array([-1.62779052,  3.48722906, -3.31269299])

stop 조건이 완화되었기 때문에 tolerance값이 매우 작았을 때보다 (stop 조건이 더 강화되었을 때) 시간은 덜 소요되지만, 찾은 최적값이 global minimum loss일지는 정확하지 않다. (abs(new_loss-loss)가 0만큼의 작은 값으로 수렴하지는 않았기 때문이다.)

In [76]:
new_param_sgd1 = gradient_descent(X_train, y_train, learning_rate=0.05, max_iter=100000, tolerance=0.0001, optimizer="sgd")
new_param_sgd1

10000 loss:  0.2990786420755581 params:  [-1.87042734  4.07452511 -3.77693438] gradients:  [0.27634761705500155, 0.2586999652825899, 0.21266396470759003]
20000 loss:  0.3127763003691973 params:  [-1.60278544  3.86780903 -4.1802995 ] gradients:  [-0.07430560459481328, -0.12515934138776924, -0.04247809755799006]


array([-1.60278544,  3.86780903, -4.1802995 ])

learning_rate가 작았을 때보다 gradients 값이 커졌다. (학습률이 높아지면서 어디로 튈지 모르는 불안정성이 더 커졌기 때문인 것으로 생각된다.)

## Predict Label

In [77]:
y_predict = []
for i in range(len(y_test)):
    p = logistic(X_test.iloc[i,:], new_param_bgd) #Batch Gradient Descent로 추정한 parameter 사용
    if p> 0.5 :
        y_predict.append(1)
    else :
        y_predict.append(0)

## Confusion Matrix

In [78]:
from sklearn.metrics import *
tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
confusion_matrix(y_test, y_predict)

array([[38,  2],
       [ 1,  9]], dtype=int64)

In [79]:
print(classification_report(y_test, y_predict, target_names=['class 0', 'class 1']))

              precision    recall  f1-score   support

     class 0       0.97      0.95      0.96        40
     class 1       0.82      0.90      0.86        10

    accuracy                           0.94        50
   macro avg       0.90      0.93      0.91        50
weighted avg       0.94      0.94      0.94        50



Batch Gradient Descent로 추정한 parameter를 이용했을 때의 정확도는 0.94로 꽤 좋은 편이다. (precision값을 봐도 같은 결론을 내릴 수 있다.)